## Importing necessary files

In [1]:
import pandas as pd
import numpy as np

# Reading train data and cleaning it  

In [2]:
df=pd.read_csv(r"/kaggle/input/solar-panel-degradation/dataset/train.csv")
df.head()

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,id,temperature,irradiance,humidity,panel_age,maintenance_count,soiling_ratio,voltage,current,module_temperature,cloud_coverage,wind_speed,pressure,string_id,error_code,installation_type,efficiency
0,0,7.817315,576.179270,41.24308670850264,32.135501,4.0,0.803199,37.403527,1.963787,13.691147,62.494044,12.82491203459621,1018.8665053152533,A1,NaN,NaN,0.562096
1,1,24.785727,240.003973,1.3596482765960705,19.977460,8.0,0.479456,21.843315,0.241473,27.545096,43.851238,12.012043660984917,1025.6238537572883,D4,E00,dual-axis,0.396447
2,2,46.652695,687.612799,91.26536837560256,1.496401,4.0,0.822398,48.222882,4.191800,43.363708,NaN,1.814399755560454,1010.9226539809573,C3,E00,NaN,0.573776
3,3,53.339567,735.141179,96.19095521176159,18.491582,3.0,0.837529,46.295748,0.960567,57.720436,67.361473,8.736258932034128,1021.8466633134253,A1,NaN,dual-axis,0.629009
4,4,5.575374,12.241203,27.495073003585226,30.722697,6.0,0.551833,0.000000,0.898062,6.786263,3.632000,0.52268384077164,1008.5559577591927,B2,E00,fixed,0.341874


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 17 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   id                  20000 non-null  int64  
 1   temperature         18999 non-null  float64
 2   irradiance          19013 non-null  float64
 3   humidity            20000 non-null  object 
 4   panel_age           18989 non-null  float64
 5   maintenance_count   18973 non-null  float64
 6   soiling_ratio       18990 non-null  float64
 7   voltage             19007 non-null  float64
 8   current             19023 non-null  float64
 9   module_temperature  19022 non-null  float64
 10  cloud_coverage      18990 non-null  float64
 11  wind_speed          20000 non-null  object 
 12  pressure            20000 non-null  object 
 13  string_id           20000 non-null  object 
 14  error_code          14088 non-null  object 
 15  installation_type   14972 non-null  object 
 16  effi

In [4]:
df["humidity"] = pd.to_numeric(df["humidity"], errors='coerce')
df["wind_speed"] = pd.to_numeric(df["wind_speed"], errors='coerce')
df["pressure"] = pd.to_numeric(df["pressure"], errors='coerce')


In [5]:
mode_value = df['installation_type'].mode()[0]
df['installation_type'].fillna(mode_value, inplace=True)
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
df['installation_type'] = le.fit_transform(df['installation_type'])

/tmp/ipykernel_35/776503627.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['installation_type'].fillna(mode_value, inplace=True)


In [6]:
columns_to_fill = ['irradiance','wind_speed',"pressure","humidity", 'soiling_ratio', 'voltage', 'current',"maintenance_count","temperature","cloud_coverage","module_temperature","panel_age"]

for col in columns_to_fill:
    df[col].fillna(df[col].mean(), inplace=True)

/tmp/ipykernel_35/999266841.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(df[col].mean(), inplace=True)


In [8]:
df['irradiance'] = df['irradiance'].where((df['irradiance'] >= 0) & (df['irradiance'] <= 1500), df['irradiance'].median())
df['humidity'] = df['humidity'].where((df['humidity'] >= 70) , df['humidity'].median())
df['soiling_ratio'] = df['soiling_ratio'].where((df['soiling_ratio'] <= 70) , df['soiling_ratio'].median())

# Choosing relevant features, using MI,Correlation, VIF

In [9]:
df=df.drop(["string_id","error_code","id"], axis=1)
correlation_matrix=df.corr()
print("correlation_matrix", correlation_matrix)

correlation_matrix                     temperature  irradiance  humidity  panel_age  \
temperature            1.000000    0.007529  0.000960  -0.008875   
irradiance             0.007529    1.000000 -0.002802   0.008322   
humidity               0.000960   -0.002802  1.000000  -0.007296   
panel_age             -0.008875    0.008322 -0.007296   1.000000   
maintenance_count     -0.002227   -0.007113  0.002730  -0.003765   
soiling_ratio          0.006618   -0.001180 -0.001343   0.000371   
voltage               -0.008855    0.236733 -0.000451   0.011151   
current                0.006706    0.436621 -0.010382   0.005912   
module_temperature     0.867254    0.006848  0.000217  -0.006256   
cloud_coverage         0.003607    0.001001  0.002749   0.000368   
wind_speed            -0.005189   -0.002674  0.004155   0.011885   
pressure              -0.003224   -0.006786  0.008385   0.002000   
installation_type     -0.001855    0.001485  0.009912  -0.000506   
efficiency            -0.0428

In [10]:
import pandas as pd
#from sklearn.feature_selection import mutual_info_classif  # for classification
from sklearn.feature_selection import mutual_info_regression  # for regression
from sklearn.preprocessing import LabelEncoder

# Separate features and target
X = df.drop('efficiency', axis=1)
y = df['efficiency']

# Encode categorical features if they exist
#X = pd.get_dummies(X)

# Compute mutual information
mi_scores = mutual_info_regression(X, y)  # use mutual_info_regression for regression tasks

# Create a DataFrame for better readability
mi_df = pd.DataFrame({'Feature': X.columns, 'MI Score': mi_scores})
mi_df = mi_df.sort_values(by='MI Score', ascending=False)

print(mi_df)

               Feature  MI Score
1           irradiance  0.421359
5        soiling_ratio  0.107322
7              current  0.063498
3            panel_age  0.042320
6              voltage  0.022417
10          wind_speed  0.010987
8   module_temperature  0.007420
11            pressure  0.005480
4    maintenance_count  0.004134
2             humidity  0.003693
9       cloud_coverage  0.001666
0          temperature  0.000000
12   installation_type  0.000000


In [11]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant
import pandas as pd

X = add_constant(df[columns_to_fill])
vif = pd.DataFrame()
vif["feature"] = X.columns
vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]


In [12]:
df=df.drop(["installation_type","temperature","pressure","maintenance_count","module_temperature","wind_speed","cloud_coverage"],axis=1)

# Training different ML models
<The best one being the ensemble Technique>

In [19]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, mean_squared_error

# Step 3: Encode categorical features (if any)
df = pd.get_dummies(df)

# Step 4: Separate features and target
Z = df.drop("efficiency", axis=1)  # replace with your target
r = df["efficiency"]
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
Z_scaled = scaler.fit_transform(Z)

# Step 5: Train-test split
X_train, X_test, y_train, y_test = train_test_split(Z_scaled, r, test_size=0.3, random_state=20)

# Step 6: Create and train the XGBoost model
model = xgb.XGBRegressor()  # Use XGBRegressor() for regression
model.fit(X_train, y_train)

# Step 7: Predictions
y_pred = model.predict(X_test)
# For regression (if using XGBRegressor)
print("RMSE:", mean_squared_error(y_test, y_pred, squared=False))
Score = 100*(1-np.sqrt(mean_squared_error(y_test,y_pred)))
print("score:", Score)

RMSE: 0.11098600530447594
score: 88.9013994695524


In [20]:
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor(random_state=4,n_estimators=1200)  # or RandomForestRegressor()
model.fit(X_train, y_train)

# Step 7: Make predictions
y_pred = model.predict(X_test)
print("RMSE:", mean_squared_error(y_test, y_pred, squared=False))
Score = 100*(1-np.sqrt(mean_squared_error(y_test,y_pred)))
print("score:", Score)

RMSE: 0.10751224139884206
score: 89.24877586011579


In [21]:
from sklearn.svm import SVR  # Support Vector Regressor
model1 = SVR(kernel='rbf',gamma='auto',C=0.15)
model1.fit(X_train, y_train)
y_pred = model1.predict(X_test)
print("RMSE:", mean_squared_error(y_test, y_pred, squared=False))
Score = 100*(1-np.sqrt(mean_squared_error(y_test,y_pred)))
print("score:", Score)

RMSE: 0.10665690480333943
score: 89.33430951966605


In [22]:
import numpy as np
# gradient bossting regressor.
from sklearn.ensemble import GradientBoostingRegressor
est = GradientBoostingRegressor(
    n_estimators=1200, learning_rate=0.01, max_depth=3, random_state=50,
    loss='squared_error'
)
est = est.fit(X_train, y_train)
y_pred = est.predict(X_test)
Score = 100*(1-np.sqrt(mean_squared_error(y_test,y_pred)))
print("score:", Score)


score: 89.33430951966605


In [24]:
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error, r2_score

model = CatBoostRegressor(verbose=0, iterations=1000, learning_rate=0.01, depth=7, random_state=200)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

print("RMSE:", mean_squared_error(y_test, y_pred, squared=False))
print("R² Score:", r2_score(y_test, y_pred))
Score = 100*(1-np.sqrt(mean_squared_error(y_test,y_pred)))
print("score:", Score)


RMSE: 0.10414192457478555
R² Score: 0.4424400359194032
score: 89.58580754252145


In [25]:
from sklearn.ensemble import VotingRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR 
from sklearn.ensemble import RandomForestRegressor


reg1 = GradientBoostingRegressor(random_state=10,n_estimators=1200, learning_rate=0.01, max_depth=3, loss='squared_error')
reg2 = RandomForestRegressor(random_state=1,n_estimators=1200)
reg3 = CatBoostRegressor(verbose=0, iterations=1000, learning_rate=0.01, depth=7, random_state=2)
reg4 = SVR(kernel='rbf',gamma='auto',C=0.15)


ereg = VotingRegressor(estimators=[('gb', reg1), ('rf', reg2),('cb',reg3),('sv',reg4)])
ereg = ereg.fit(X_train, y_train)
y_pred = ereg.predict(X_test)
print("RMSE:", mean_squared_error(y_test, y_pred, squared=False))
Score = 100*(1-np.sqrt(mean_squared_error(y_test,y_pred)))
print("score:", Score)

RMSE: 0.10454675570321396
score: 89.5453244296786


In [26]:
class ELMRegressor:
    def __init__(self, n_hidden=100, activation='sigmoid',random_state=None):
        self.n_hidden = n_hidden
        self.activation = activation

    def _activate(self, X):
        if self.activation == 'sigmoid':
            return 1 / (1 + np.exp(-X))
        elif self.activation == 'tanh':
            return np.tanh(X)
        elif self.activation == 'relu':
            return np.maximum(0, X)
        else:
            raise ValueError("Unsupported activation")

    def fit(self, X, y):
        n_samples, n_features = X.shape

        # Random weights and biases
        self.input_weights = np.random.randn(n_features, self.n_hidden)
        self.biases = np.random.randn(self.n_hidden)

        # Hidden layer output
        H = self._activate(np.dot(X, self.input_weights) + self.biases)

        # Moore-Penrose pseudoinverse to solve output weights
        self.output_weights = np.dot(np.linalg.pinv(H), y)

    def predict(self, X):
        H = self._activate(np.dot(X, self.input_weights) + self.biases)
        return np.dot(H, self.output_weights)


In [27]:
elm = ELMRegressor(n_hidden=500, activation='sigmoid')
elm.fit(X_train, y_train)

# Predict
y_pred_scaled = elm.predict(X_test)
# Evaluation
print("RMSE:", mean_squared_error(y_test, y_pred, squared=False))
Score = 100*(1-np.sqrt(mean_squared_error(y_test,y_pred)))
print("score:", Score)


RMSE: 0.10454675570321396
score: 89.5453244296786


# Reading test data for final prediction

In [ ]:
df_test=pd.read_csv("/kaggle/input/solar-panel-degradation/dataset/test.csv")

In [ ]:
df_test["humidity"] = pd.to_numeric(df_test["humidity"], errors='coerce')
df_test["wind_speed"] = pd.to_numeric(df_test["wind_speed"], errors='coerce')
df_test["pressure"] = pd.to_numeric(df_test["pressure"], errors='coerce')

In [ ]:
df_test=df_test.drop(["string_id",'cloud_coverage',"module_temperature","maintenance_count","wind_speed","error_code","id","temperature","pressure","installation_type"], axis=1)

In [ ]:
columns_to_fill = ['irradiance',"humidity", 'soiling_ratio', 'voltage', 'current',"panel_age"]

for col in columns_to_fill:
    df_test[col].fillna(df_test[col].median(), inplace=True)

In [ ]:
df_test['irradiance'] = df_test['irradiance'].where((df_test['irradiance'] >= 0) & (df_test['irradiance'] <= 1500), df_test['irradiance'].median())
df_test['humidity'] = df_test['humidity'].where((df_test['humidity'] >= 70) , df_test['humidity'].median())
df['soiling_ratio'] = df['soiling_ratio'].where((df['soiling_ratio'] <= 70) , df_test['soiling_ratio'].median())

In [ ]:
df_test_1=pd.read_csv("/kaggle/input/solar-panel-degradation/dataset/test.csv")

In [ ]:
X_scaled1 = scaler.fit_transform(df_test)
y_test_pred1 =ereg.predict(X_scaled1)

In [ ]:
df_test["efficiency"]=y_test_pred1
df_test["id"]=df_test_1["id"]

In [ ]:
z=df_test[["id","efficiency"]]

In [ ]:
z.to_csv("submission1.csv", index=False)

 Note:-All the models were equally good, but ensemble using voting regressor showed best results